In [ ]:
## This file implements neural networks with/without dropout and regularizer for p002ypresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for each model.

In [91]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [92]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/p002ypresabsSTCC_qual.csv')
df.shape

(253, 2035)

In [93]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [94]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [95]:
df.head()

,id,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,...,group_8644,group_8645,group_8646,group_8815,group_8892,group_9007,group_9104,ST,CC,pheno
0,107,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0
1,109,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,120335,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0
4,120337,0,1,1,1,1,0,0,1,1,...,0,0,0,0,0,0,0,5,5,0


In [96]:
df['pheno'].value_counts()

0    220
1     30
2      3
Name: pheno, dtype: int64

In [97]:
df_clean = df.drop(columns=['id'])

In [98]:
df_clean.shape

(253, 2034)

In [99]:
df_clean.head()

,TTTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTA,TTTTTTTATGAAT,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTGACTAAAATTAATGAAAAGTGAAAATAGTATTGGAACTCAATATCTTTAATGATTTAATGAATAATTTTTATTGAAAGCGATAATTCGTATTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCATTAGT,TTTTTTCATTAGTAA,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,...,group_8644,group_8645,group_8646,group_8815,group_8892,group_9007,group_9104,ST,CC,pheno
0,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0
1,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,8,8,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,1
3,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0
4,0,1,1,1,1,0,0,1,1,0,...,0,0,0,0,0,0,0,5,5,0


In [100]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 2034) (253,)


In [101]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 220), (1, 220), (2, 220)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [102]:
############# Fully-Connected Neural Network ################

In [103]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [104]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [105]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [106]:
dat

,0,test
0,CFBRSa26,0
1,NRS109,2
2,NRS112,0
3,NRS216,1
4,NRS021,0
...,...,...
193,CFBREBSa133,0
194,NRS209,2
195,NRS109,2
196,NRS209,2


In [107]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 481us/step - loss: 1.0873 - accuracy: 0.6385 - val_loss: 0.7128 - val_accuracy: 0.7222
Epoch 2/100
462/462 [==============================] - 0s 349us/step - loss: 0.5253 - accuracy: 0.7879 - val_loss: 0.6271 - val_accuracy: 0.8384
Epoch 3/100
462/462 [==============================] - 0s 268us/step - loss: 0.4493 - accuracy: 0.8268 - val_loss: 0.3268 - val_accuracy: 0.8636
Epoch 4/100
462/462 [==============================] - 0s 255us/step - loss: 0.3400 - accuracy: 0.8615 - val_loss: 0.3445 - val_accuracy: 0.8636
Epoch 5/100
462/462 [==============================] - 0s 283us/step - loss: 0.3445 - accuracy: 0.8615 - val_loss: 0.3043 - val_accuracy: 0.8788
Epoch 6/100
462/462 [==============================] - 0s 223us/step - loss: 0.3267 - accuracy: 0.8550 - val_loss: 0.2836 - val_accuracy: 0.8434
Epoch 7/100
462/462 [==============================] - 0s 218us/step - loss: 0.2765 

Epoch 57/100
462/462 [==============================] - 0s 333us/step - loss: 0.0365 - accuracy: 0.9913 - val_loss: 0.0562 - val_accuracy: 0.9949
Epoch 58/100
462/462 [==============================] - 0s 502us/step - loss: 0.0370 - accuracy: 0.9913 - val_loss: 0.0575 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 557us/step - loss: 0.0345 - accuracy: 0.9935 - val_loss: 0.0891 - val_accuracy: 0.9747
Epoch 60/100
462/462 [==============================] - 0s 414us/step - loss: 0.0457 - accuracy: 0.9870 - val_loss: 0.0475 - val_accuracy: 1.0000
Epoch 61/100
462/462 [==============================] - 0s 395us/step - loss: 0.0364 - accuracy: 0.9935 - val_loss: 0.0470 - val_accuracy: 0.9949
Epoch 62/100
462/462 [==============================] - 0s 322us/step - loss: 0.0372 - accuracy: 0.9913 - val_loss: 0.0575 - val_accuracy: 0.9747
Epoch 63/100
462/462 [==============================] - 0s 254us/step - loss: 0.0325 - accuracy: 0.9913 - val_loss: 0.0537 -

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.129369). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


462/462 [==============================] - 0s 312us/step - loss: 0.0174 - accuracy: 0.9978 - val_loss: 0.0377 - val_accuracy: 0.9949
Epoch 91/100
462/462 [==============================] - 0s 237us/step - loss: 0.0157 - accuracy: 1.0000 - val_loss: 0.0433 - val_accuracy: 0.9949
Epoch 92/100
462/462 [==============================] - 0s 222us/step - loss: 0.0158 - accuracy: 0.9978 - val_loss: 0.0398 - val_accuracy: 0.9949
Epoch 93/100
462/462 [==============================] - 0s 267us/step - loss: 0.0159 - accuracy: 1.0000 - val_loss: 0.0361 - val_accuracy: 0.9949
Epoch 94/100
462/462 [==============================] - 0s 265us/step - loss: 0.0151 - accuracy: 0.9978 - val_loss: 0.0385 - val_accuracy: 0.9949
Epoch 95/100
462/462 [==============================] - 0s 241us/step - loss: 0.0207 - accuracy: 0.9957 - val_loss: 0.0402 - val_accuracy: 0.9949
Epoch 96/100
462/462 [==============================] - 0s 220us/step - loss: 0.0290 - accuracy: 0.9957 - val_loss: 0.0428 - val_accuracy

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

198/198 [==============================] - 0s 147us/step
over-sampling test accuracy: 97.47%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 2, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 2, 2, 1, 0, 0, 2, 0, 2, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 0, 0, 0, 2, 0, 0, 1, 2, 2, 0, 0, 1,
       0, 1, 0, 2, 0, 0, 1, 1, 2, 1, 1, 1, 2, 0, 1, 2, 0, 0, 2, 1, 1, 0,
       0, 2, 0, 0, 2, 0, 1, 2, 1, 1, 1, 2, 0, 2, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 0, 0,
       2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 0, 2, 1, 0, 2, 0, 0, 2, 2, 0, 2, 0,
       0, 2, 0, 2, 1, 0, 2, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 0, 2, 2, 2, 1])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,CFBRSa26,0,0
1,NRS109,2,2
2,NRS112,0,0
3,NRS216,1,1
4,NRS021,0,0
...,...,...,...
193,CFBREBSa133,0,0
194,NRS209,2,2
195,NRS109,2,2
196,NRS209,2,2


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,9.999528e-01,4.725889e-05,3.082832e-09
1,1.021427e-03,7.837663e-04,9.981949e-01
2,9.970143e-01,2.985639e-03,1.549940e-07
3,2.931194e-02,9.706042e-01,8.391447e-05
4,9.988996e-01,1.100384e-03,1.590173e-10
...,...,...,...
193,1.000000e+00,7.373644e-10,7.920553e-09
194,2.044007e-07,4.704020e-04,9.995294e-01
195,1.021427e-03,7.837656e-04,9.981949e-01
196,2.044007e-07,4.704020e-04,9.995294e-01


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [26]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1p002ypST.csv", index = False,
         header=None)

In [25]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 271us/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.0436 - val_accuracy: 0.9899
Epoch 2/100
462/462 [==============================] - 0s 248us/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 0.0378 - val_accuracy: 0.9949
Epoch 3/100
462/462 [==============================] - 0s 195us/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 0.0372 - val_accuracy: 0.9747
Epoch 4/100
462/462 [==============================] - 0s 191us/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.0332 - val_accuracy: 0.9949
Epoch 5/100
462/462 [==============================] - 0s 195us/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.0424 - val_accuracy: 0.9747
Epoch 6/100
462/462 [==============================] - 0s 196us/step - loss: 0.0125 - accuracy: 1.0000 - val_loss: 0.0325 - val_accuracy: 0.9949
Epoch 7/100
462/462 [==============================] - 0s 200us/step - loss: 0.0107 

Epoch 57/100
462/462 [==============================] - 0s 252us/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0366 - val_accuracy: 0.9747
Epoch 58/100
462/462 [==============================] - 0s 177us/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0388 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 164us/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0460 - val_accuracy: 0.9798
Epoch 60/100
462/462 [==============================] - 0s 172us/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0348 - val_accuracy: 0.9949
Epoch 61/100
462/462 [==============================] - 0s 170us/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.0501 - val_accuracy: 0.9798
Epoch 62/100
462/462 [==============================] - 0s 179us/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.0395 - val_accuracy: 0.9798
Epoch 63/100
462/462 [==============================] - 0s 175us/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0371 -

In [26]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 99.99%


In [108]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [109]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [110]:
y_prob = df_proba[df_proba['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[9.99952800e-01, 4.72588870e-05, 3.08283150e-09],
       [1.02142730e-03, 7.83766300e-04, 9.98194900e-01],
       [9.97014300e-01, 2.98563930e-03, 1.54993970e-07],
       [2.93119440e-02, 9.70604200e-01, 8.39144700e-05],
       [9.98899600e-01, 1.10038390e-03, 1.59017260e-10],
       [9.99733500e-01, 2.66428570e-04, 8.31315800e-09],
       [1.63454510e-06, 2.06930840e-03, 9.97929100e-01],
       [1.63454510e-06, 2.06930840e-03, 9.97929100e-01],
       [1.02142730e-03, 7.83766300e-04, 9.98194900e-01],
       [1.00000000e+00, 1.90122370e-08, 1.44964470e-12],
       [1.53664180e-02, 9.84576900e-01, 5.66871530e-05],
       [9.99995230e-01, 4.72467600e-06, 3.18319260e-13],
       [8.15281900e-02, 9.18467500e-01, 4.31467700e-06],
       [4.21984160e-02, 9.42683340e-01, 1.51182730e-02],
       [9.98228250e-01, 1.77174710e-03, 3.35112600e-10],
       [1.72360500e-02, 9.82735500e-01, 2.83248120e-05],
       [1.63454510e-06, 2.06930840e-03, 9.97929100e-01],
       [1.22267720e-03, 9.98767

In [111]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [112]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9996939087848178

In [113]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [114]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9996939087848178

In [115]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [116]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [117]:
dat2

,0,test
0,NRS109,2
1,NRS109,2
2,NRS222,0
3,NRS109,2
4,GA50245,0
...,...,...
193,NRS148,2
194,NRS266,1
195,NRS109,2
196,NRS149,0


In [118]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [31]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [32]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 421us/step - loss: 1.3324 - accuracy: 0.6645 - val_loss: 0.6076 - val_accuracy: 0.8131
Epoch 2/100
462/462 [==============================] - 0s 244us/step - loss: 0.6414 - accuracy: 0.7944 - val_loss: 0.4623 - val_accuracy: 0.8030
Epoch 3/100
462/462 [==============================] - 0s 310us/step - loss: 0.3956 - accuracy: 0.8377 - val_loss: 0.3848 - val_accuracy: 0.8333
Epoch 4/100
462/462 [==============================] - 0s 365us/step - loss: 0.3238 - accuracy: 0.8680 - val_loss: 0.3237 - val_accuracy: 0.8586
Epoch 5/100
462/462 [==============================] - 0s 348us/step - loss: 0.2690 - accuracy: 0.9004 - val_loss: 0.3634 - val_accuracy: 0.8838
Epoch 6/100
462/462 [==============================] - 0s 260us/step - loss: 0.3056 - accuracy: 0.8918 - val_loss: 0.3179 - val_accuracy: 0.8990
Epoch 7/100
462/462 [==============================] - 0s 242us/step - loss: 0.2251 

Epoch 57/100
462/462 [==============================] - 0s 185us/step - loss: 0.0396 - accuracy: 0.9892 - val_loss: 0.1040 - val_accuracy: 0.9798
Epoch 58/100
462/462 [==============================] - 0s 227us/step - loss: 0.0303 - accuracy: 0.9978 - val_loss: 0.0744 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 233us/step - loss: 0.0263 - accuracy: 0.9957 - val_loss: 0.0880 - val_accuracy: 0.9747
Epoch 60/100
462/462 [==============================] - 0s 205us/step - loss: 0.0292 - accuracy: 0.9957 - val_loss: 0.0840 - val_accuracy: 0.9747
Epoch 61/100
462/462 [==============================] - 0s 204us/step - loss: 0.0265 - accuracy: 0.9957 - val_loss: 0.0779 - val_accuracy: 0.9899
Epoch 62/100
462/462 [==============================] - 0s 183us/step - loss: 0.0239 - accuracy: 0.9978 - val_loss: 0.1004 - val_accuracy: 0.9646
Epoch 63/100
462/462 [==============================] - 0s 186us/step - loss: 0.0258 - accuracy: 0.9957 - val_loss: 0.1055 -

In [37]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

198/198 [==============================] - 0s 173us/step
over-sampling test accuracy: 97.47%


In [34]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([2, 2, 0, 2, 0, 2, 0, 1, 0, 2, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 1,
       2, 1, 1, 2, 1, 2, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 2, 1, 0, 0, 2, 1,
       2, 0, 0, 2, 1, 1, 1, 2, 0, 1, 1, 2, 2, 0, 2, 0, 0, 1, 1, 2, 0, 2,
       1, 1, 0, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 0, 1, 1, 0, 1, 2, 0, 1, 0,
       2, 0, 2, 1, 1, 1, 2, 2, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1,
       0, 2, 1, 1, 1, 1, 0, 0, 0, 2, 0, 2, 1, 2, 1, 2, 2, 2, 1, 0, 1, 1,
       1, 1, 2, 2, 0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 1, 2, 2, 2, 0, 2, 1, 0,
       0, 2, 2, 2, 1, 1, 0, 2, 1, 2, 0, 0, 2, 0, 2, 2, 1, 1, 1, 2, 1, 2,
       0, 1, 1, 1, 2, 1, 2, 1, 0, 0, 0, 0, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2])

In [35]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,NRS109,2,2
1,NRS109,2,2
2,NRS222,0,0
3,NRS109,2,2
4,GA50245,0,0
...,...,...,...
193,NRS148,2,2
194,NRS266,1,1
195,NRS109,2,2
196,NRS149,0,1


In [36]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [37]:
dat_proba2

,0,1,2
0,6.302851e-04,0.005169,9.942011e-01
1,6.302851e-04,0.005169,9.942011e-01
2,9.947300e-01,0.005270,3.008131e-07
3,6.302851e-04,0.005169,9.942011e-01
4,9.987645e-01,0.001235,6.272662e-07
...,...,...,...
193,1.341323e-09,0.001519,9.984811e-01
194,4.687704e-04,0.999517,1.455117e-05
195,6.302848e-04,0.005169,9.942011e-01
196,4.454741e-01,0.554522,3.477236e-06


In [38]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [39]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2p002ypST.csv", index = False,
         header=None)

In [41]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 284us/step - loss: 0.0090 - accuracy: 0.9978 - val_loss: 0.0888 - val_accuracy: 0.9747
Epoch 2/100
462/462 [==============================] - 0s 242us/step - loss: 0.0184 - accuracy: 0.9957 - val_loss: 0.0884 - val_accuracy: 0.9747
Epoch 3/100
462/462 [==============================] - 0s 254us/step - loss: 0.0193 - accuracy: 0.9957 - val_loss: 0.0972 - val_accuracy: 0.9747
Epoch 4/100
462/462 [==============================] - 0s 245us/step - loss: 0.0143 - accuracy: 1.0000 - val_loss: 0.0881 - val_accuracy: 0.9747
Epoch 5/100
462/462 [==============================] - 0s 257us/step - loss: 0.0097 - accuracy: 0.9978 - val_loss: 0.1152 - val_accuracy: 0.9747
Epoch 6/100
462/462 [==============================] - 0s 345us/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.0959 - val_accuracy: 0.9747
Epoch 7/100
462/462 [==============================] - 0s 244us/step - loss: 0.0076 

Epoch 57/100
462/462 [==============================] - 0s 297us/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.0890 - val_accuracy: 0.9747
Epoch 58/100
462/462 [==============================] - 0s 469us/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0931 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 317us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0970 - val_accuracy: 0.9747
Epoch 60/100
462/462 [==============================] - 0s 233us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.1038 - val_accuracy: 0.9747
Epoch 61/100
462/462 [==============================] - 0s 197us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.1005 - val_accuracy: 0.9747
Epoch 62/100
462/462 [==============================] - 0s 221us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.1070 - val_accuracy: 0.9747
Epoch 63/100
462/462 [==============================] - 0s 347us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.1007 -

In [42]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 99.98%


In [119]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [120]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [121]:
y_prob2 = df_proba2[df_proba2['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[6.30285070e-04, 5.16860600e-03, 9.94201100e-01],
       [6.30285070e-04, 5.16860600e-03, 9.94201100e-01],
       [9.94730000e-01, 5.26959640e-03, 3.00813100e-07],
       [6.30285070e-04, 5.16860600e-03, 9.94201100e-01],
       [9.98764500e-01, 1.23492540e-03, 6.27266160e-07],
       [6.30285070e-04, 5.16860600e-03, 9.94201100e-01],
       [9.97446060e-01, 2.55394980e-03, 4.57869300e-09],
       [3.06966760e-03, 9.96923600e-01, 6.64078650e-06],
       [9.99999760e-01, 2.14201380e-07, 2.15502250e-11],
       [6.30285070e-04, 5.16860600e-03, 9.94201100e-01],
       [9.99815170e-01, 1.84799180e-04, 1.10494206e-07],
       [5.33969300e-04, 9.99464450e-01, 1.60185440e-06],
       [9.94723700e-01, 5.27623060e-03, 1.59178510e-07],
       [2.66609780e-06, 9.89390300e-01, 1.06069840e-02],
       [1.27158970e-03, 9.98695900e-01, 3.25350300e-05],
       [1.34133350e-09, 1.51889340e-03, 9.98481100e-01],
       [4.48031240e-05, 9.99951840e-01, 3.29720610e-06],
       [4.23477170e-10, 4.77588

In [122]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9949877563513927

In [123]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9949877563513927

In [124]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [125]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [126]:
dat3

,0,test
0,CFBREBSa110,0
1,CFBREBSa131,1
2,NRS148,2
3,NRS169,1
4,NRS073,0
...,...,...
193,NRS001,1
194,NRS191,0
195,NRS207,0
196,CA26,0


In [127]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [44]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [45]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [46]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 423us/step - loss: 1.6758 - accuracy: 0.5952 - val_loss: 0.7575 - val_accuracy: 0.6364
Epoch 2/100
462/462 [==============================] - 0s 236us/step - loss: 0.5447 - accuracy: 0.7684 - val_loss: 0.5952 - val_accuracy: 0.7626
Epoch 3/100
462/462 [==============================] - 0s 234us/step - loss: 0.4132 - accuracy: 0.8398 - val_loss: 0.5061 - val_accuracy: 0.7929
Epoch 4/100
462/462 [==============================] - 0s 285us/step - loss: 0.3633 - accuracy: 0.8615 - val_loss: 0.4361 - val_accuracy: 0.8283
Epoch 5/100
462/462 [==============================] - 0s 390us/step - loss: 0.3313 - accuracy: 0.8745 - val_loss: 0.3726 - val_accuracy: 0.8586
Epoch 6/100
462/462 [==============================] - 0s 325us/step - loss: 0.3297 - accuracy: 0.8723 - val_loss: 0.3663 - val_accuracy: 0.8283
Epoch 7/100
462/462 [==============================] - 0s 354us/step - loss: 0.2460 

Epoch 57/100
462/462 [==============================] - 0s 179us/step - loss: 0.0249 - accuracy: 0.9978 - val_loss: 0.1634 - val_accuracy: 0.9545
Epoch 58/100
462/462 [==============================] - 0s 186us/step - loss: 0.0275 - accuracy: 0.9978 - val_loss: 0.0946 - val_accuracy: 0.9747
Epoch 59/100
462/462 [==============================] - 0s 233us/step - loss: 0.0225 - accuracy: 0.9978 - val_loss: 0.0983 - val_accuracy: 0.9747
Epoch 60/100
462/462 [==============================] - 0s 465us/step - loss: 0.0232 - accuracy: 0.9978 - val_loss: 0.0936 - val_accuracy: 0.9848
Epoch 61/100
462/462 [==============================] - 0s 230us/step - loss: 0.0285 - accuracy: 0.9957 - val_loss: 0.0971 - val_accuracy: 0.9798
Epoch 62/100
462/462 [==============================] - 0s 286us/step - loss: 0.0241 - accuracy: 0.9935 - val_loss: 0.0853 - val_accuracy: 0.9848
Epoch 63/100
462/462 [==============================] - 0s 372us/step - loss: 0.0252 - accuracy: 0.9978 - val_loss: 0.0858 -

In [50]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

198/198 [==============================] - 0s 187us/step
over-sampling test accuracy: 97.47%


In [47]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([0, 1, 2, 1, 0, 1, 1, 2, 1, 1, 0, 2, 1, 1, 0, 2, 0, 2, 0, 2, 0, 2,
       1, 0, 1, 2, 2, 2, 0, 2, 2, 1, 1, 0, 1, 2, 1, 0, 0, 2, 2, 1, 0, 0,
       1, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 1,
       0, 1, 1, 2, 1, 1, 0, 1, 1, 2, 2, 1, 1, 2, 2, 0, 2, 0, 0, 1, 0, 2,
       2, 1, 2, 1, 2, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 0, 2, 0, 2, 0, 1,
       2, 2, 0, 1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 2, 0,
       1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 1, 1, 2,
       1, 0, 0, 2, 2, 0, 1, 2, 2, 2, 2, 1, 1, 0, 2, 0, 1, 2, 0, 1, 0, 1,
       1, 1, 2, 0, 2, 2, 2, 0, 0, 1, 2, 0, 1, 2, 0, 2, 1, 1, 0, 0, 0, 0])

In [48]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,CFBREBSa110,0,0
1,CFBREBSa131,1,1
2,NRS148,2,2
3,NRS169,1,1
4,NRS073,0,0
...,...,...,...
193,NRS001,1,1
194,NRS191,0,0
195,NRS207,0,0
196,CA26,0,0


In [49]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [50]:
dat_proba3

,0,1,2
0,0.999314,0.000686,3.761691e-09
1,0.036356,0.963644,1.427160e-09
2,0.000001,0.002898,9.971011e-01
3,0.015282,0.979220,5.497806e-03
4,0.999118,0.000882,1.210949e-12
...,...,...,...
193,0.009724,0.990276,5.089228e-08
194,0.845824,0.154172,4.374536e-06
195,0.999702,0.000297,7.595697e-08
196,0.999991,0.000009,1.380664e-09


In [51]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [52]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3p002ypST.csv", index = False,
         header=None)

In [54]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 277us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0910 - val_accuracy: 0.9798
Epoch 2/100
462/462 [==============================] - 0s 238us/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0869 - val_accuracy: 0.9798
Epoch 3/100
462/462 [==============================] - 0s 220us/step - loss: 0.0071 - accuracy: 0.9978 - val_loss: 0.0745 - val_accuracy: 0.9848
Epoch 4/100
462/462 [==============================] - 0s 249us/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.0924 - val_accuracy: 0.9798
Epoch 5/100
462/462 [==============================] - 0s 239us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0793 - val_accuracy: 0.9848
Epoch 6/100
462/462 [==============================] - 0s 252us/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.0761 - val_accuracy: 0.9848
Epoch 7/100
462/462 [==============================] - 0s 307us/step - loss: 0.0065 

Epoch 57/100
462/462 [==============================] - 0s 184us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0872 - val_accuracy: 0.9848
Epoch 58/100
462/462 [==============================] - 0s 210us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0847 - val_accuracy: 0.9848
Epoch 59/100
462/462 [==============================] - 0s 506us/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1055 - val_accuracy: 0.9697
Epoch 60/100
462/462 [==============================] - 0s 525us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0786 - val_accuracy: 0.9848
Epoch 61/100
462/462 [==============================] - 0s 315us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0997 - val_accuracy: 0.9697
Epoch 62/100
462/462 [==============================] - 0s 266us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0999 - val_accuracy: 0.9747
Epoch 63/100
462/462 [==============================] - 0s 269us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.0791 -

In [55]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [128]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [129]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [130]:
y_prob3 = df_proba3[df_proba3['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[9.99313830e-01, 6.86202500e-04, 3.76169140e-09],
       [3.63563600e-02, 9.63643600e-01, 1.42716030e-09],
       [1.31860800e-06, 2.89752900e-03, 9.97101100e-01],
       [1.52819770e-02, 9.79220150e-01, 5.49780620e-03],
       [9.99117700e-01, 8.82361260e-04, 1.21094910e-12],
       [1.54402840e-05, 9.99984500e-01, 8.83860100e-11],
       [7.55906150e-05, 9.77803100e-01, 2.21213330e-02],
       [1.31860800e-06, 2.89752900e-03, 9.97101100e-01],
       [5.66168600e-04, 9.99433700e-01, 7.55483640e-08],
       [3.30562800e-01, 6.69437200e-01, 1.04187174e-08],
       [9.99978540e-01, 2.14273660e-05, 2.31031800e-14],
       [1.49616090e-03, 8.12478650e-04, 9.97691400e-01],
       [2.47907300e-02, 9.75209060e-01, 2.69607230e-07],
       [9.72403900e-03, 9.90276000e-01, 5.08922820e-08],
       [9.99975900e-01, 2.05343300e-05, 3.57822070e-06],
       [1.49616090e-03, 8.12478650e-04, 9.97691400e-01],
       [9.99882700e-01, 6.67148600e-05, 5.05345740e-05],
       [1.49616090e-03, 8.12478

In [131]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.9920033670033671

In [132]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.9920033670033671

In [133]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [134]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [135]:
dat4

,0,test
0,NRS110,1
1,NRS216,1
2,NRS386,1
3,CFBRSa25,0
4,BCH-SA-03,1
...,...,...
193,NRS216,1
194,CFBREBSa110,0
195,NRS148,2
196,GA27,0


In [136]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [57]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [58]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [59]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 1ms/step - loss: 1.4792 - accuracy: 0.6342 - val_loss: 0.7244 - val_accuracy: 0.6667
Epoch 2/100
462/462 [==============================] - 0s 229us/step - loss: 0.6459 - accuracy: 0.7511 - val_loss: 0.4009 - val_accuracy: 0.8333
Epoch 3/100
462/462 [==============================] - 0s 465us/step - loss: 0.4748 - accuracy: 0.7835 - val_loss: 0.4055 - val_accuracy: 0.8434
Epoch 4/100
462/462 [==============================] - 0s 256us/step - loss: 0.3709 - accuracy: 0.8463 - val_loss: 0.3417 - val_accuracy: 0.8182
Epoch 5/100
462/462 [==============================] - 0s 330us/step - loss: 0.3220 - accuracy: 0.8680 - val_loss: 0.4860 - val_accuracy: 0.7828
Epoch 6/100
462/462 [==============================] - 0s 395us/step - loss: 0.3392 - accuracy: 0.8485 - val_loss: 0.3325 - val_accuracy: 0.9040
Epoch 7/100
462/462 [==============================] - 0s 223us/step - loss: 0.2513 - 

Epoch 57/100
462/462 [==============================] - 0s 188us/step - loss: 0.0372 - accuracy: 0.9913 - val_loss: 0.0877 - val_accuracy: 0.9747
Epoch 58/100
462/462 [==============================] - 0s 201us/step - loss: 0.0339 - accuracy: 0.9935 - val_loss: 0.1071 - val_accuracy: 0.9646
Epoch 59/100
462/462 [==============================] - 0s 265us/step - loss: 0.0329 - accuracy: 0.9935 - val_loss: 0.1059 - val_accuracy: 0.9646
Epoch 60/100
462/462 [==============================] - 0s 236us/step - loss: 0.0277 - accuracy: 0.9978 - val_loss: 0.0986 - val_accuracy: 0.9697
Epoch 61/100
462/462 [==============================] - 0s 208us/step - loss: 0.0320 - accuracy: 0.9892 - val_loss: 0.0872 - val_accuracy: 0.9697
Epoch 62/100
462/462 [==============================] - 0s 303us/step - loss: 0.0291 - accuracy: 0.9935 - val_loss: 0.0876 - val_accuracy: 0.9697
Epoch 63/100
462/462 [==============================] - 0s 225us/step - loss: 0.0267 - accuracy: 0.9935 - val_loss: 0.0890 -

In [63]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

198/198 [==============================] - 0s 291us/step
over-sampling test accuracy: 96.97%


In [60]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 1, 1, 0, 1, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 1, 0, 2, 1,
       1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 0, 0, 1, 1, 1,
       0, 1, 1, 2, 2, 0, 0, 2, 1, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 1, 2, 2,
       0, 0, 1, 2, 2, 0, 0, 1, 2, 0, 1, 2, 1, 1, 0, 1, 2, 0, 2, 0, 0, 1,
       0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 0, 0, 1, 2, 2, 1, 1, 2, 0,
       1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 0, 0, 2, 1, 1, 0, 1, 2, 2, 2, 0, 1,
       0, 2, 1, 2, 0, 0, 1, 2, 1, 2, 2, 0, 0, 2, 0, 1, 1, 0, 0, 0, 1, 2,
       2, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 2, 1, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 1, 0, 2, 0, 2])

In [61]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS110,1,1
1,NRS216,1,1
2,NRS386,1,1
3,CFBRSa25,0,0
4,BCH-SA-03,1,1
...,...,...,...
193,NRS216,1,1
194,CFBREBSa110,0,0
195,NRS148,2,2
196,GA27,0,0


In [62]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [63]:
dat_proba4

,0,1,2
0,0.002259,0.997741,8.019265e-12
1,0.023295,0.976704,2.383147e-07
2,0.038680,0.961320,1.034880e-09
3,0.999643,0.000357,1.617060e-12
4,0.010256,0.989744,5.751620e-11
...,...,...,...
193,0.023295,0.976704,2.383147e-07
194,0.999738,0.000262,7.801651e-09
195,0.000003,0.001161,9.988357e-01
196,0.999186,0.000814,2.547308e-17


In [64]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [65]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4p002ypST.csv", index = False,
         header=None)

In [67]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 285us/step - loss: 0.0099 - accuracy: 1.0000 - val_loss: 0.1331 - val_accuracy: 0.9697
Epoch 2/100
462/462 [==============================] - 0s 228us/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 0.1356 - val_accuracy: 0.9697
Epoch 3/100
462/462 [==============================] - 0s 242us/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 0.1199 - val_accuracy: 0.9697
Epoch 4/100
462/462 [==============================] - 0s 220us/step - loss: 0.0100 - accuracy: 0.9978 - val_loss: 0.1229 - val_accuracy: 0.9747
Epoch 5/100
462/462 [==============================] - 0s 229us/step - loss: 0.0085 - accuracy: 0.9978 - val_loss: 0.1394 - val_accuracy: 0.9697
Epoch 6/100
462/462 [==============================] - 0s 223us/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.1630 - val_accuracy: 0.9646
Epoch 7/100
462/462 [==============================] - 0s 210us/step - loss: 0.0077 

Epoch 57/100
462/462 [==============================] - 0s 272us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.1768 - val_accuracy: 0.9646
Epoch 58/100
462/462 [==============================] - 0s 243us/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.1419 - val_accuracy: 0.9697
Epoch 59/100
462/462 [==============================] - 0s 264us/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.1721 - val_accuracy: 0.9646
Epoch 60/100
462/462 [==============================] - 0s 358us/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.1327 - val_accuracy: 0.9697
Epoch 61/100
462/462 [==============================] - 0s 437us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.1768 - val_accuracy: 0.9646
Epoch 62/100
462/462 [==============================] - 0s 486us/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.1633 - val_accuracy: 0.9697
Epoch 63/100
462/462 [==============================] - 0s 389us/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.1316 -

In [68]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 100.00%


In [137]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [138]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [139]:
y_prob4 = df_proba4[df_proba4['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[2.25869940e-03, 9.97741340e-01, 8.01926500e-12],
       [2.32953320e-02, 9.76704500e-01, 2.38314660e-07],
       [3.86803600e-02, 9.61319700e-01, 1.03487970e-09],
       [9.99643450e-01, 3.56529520e-04, 1.61705980e-12],
       [1.02559660e-02, 9.89744000e-01, 5.75162000e-11],
       [2.25869940e-03, 9.97741340e-01, 8.01926500e-12],
       [3.23752900e-05, 8.90931600e-05, 9.99878500e-01],
       [3.23752900e-05, 8.90931600e-05, 9.99878500e-01],
       [3.23752900e-05, 8.90931600e-05, 9.99878500e-01],
       [9.99736500e-01, 2.48903100e-04, 1.45464730e-05],
       [3.02686100e-06, 1.16119230e-03, 9.98835740e-01],
       [9.82613500e-01, 1.73864960e-02, 2.57739800e-10],
       [9.95524300e-01, 4.47564850e-03, 7.19103850e-08],
       [9.71900460e-01, 2.80995740e-02, 2.64107350e-11],
       [9.99098660e-01, 9.01342840e-04, 1.45180080e-12],
       [3.23752900e-05, 8.90931600e-05, 9.99878500e-01],
       [9.99938130e-01, 6.18358900e-05, 3.06039480e-09],
       [3.23752900e-05, 8.90931

In [140]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9964799510254055

In [141]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9964799510254055

In [142]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9957912457912458

In [144]:
np.std(ovos)

0.002770321025286243

In [145]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9957912457912458

In [146]:
np.std(ovrs)

0.002770321025286243

In [69]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [70]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 97.35%


In [71]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.002186922149028386


In [72]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [73]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 99.99%


In [74]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 5.5162433e-05


In [147]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=567,
                                                    stratify=y_over)

In [148]:
dat5 = pd.DataFrame(X_test_over[:,0])
dat5['test'] = y_test_over

In [149]:
dat5

,0,test
0,NRS241,1
1,NRS148,2
2,NRS255,1
3,NRS214,0
4,NRS148,2
...,...,...
193,CFBRSa30,0
194,NRS266,1
195,SR4152,0
196,NRS109,2


In [150]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [151]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [71]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [72]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=16, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 1s 1ms/step - loss: 5.5119 - accuracy: 0.4935 - val_loss: 1.5872 - val_accuracy: 0.5707
Epoch 2/100
462/462 [==============================] - 0s 467us/step - loss: 3.5817 - accuracy: 0.5931 - val_loss: 0.9677 - val_accuracy: 0.6818
Epoch 3/100
462/462 [==============================] - 0s 634us/step - loss: 3.1734 - accuracy: 0.6255 - val_loss: 1.1601 - val_accuracy: 0.6869
Epoch 4/100
462/462 [==============================] - 0s 323us/step - loss: 3.1783 - accuracy: 0.6126 - val_loss: 0.9472 - val_accuracy: 0.6970
Epoch 5/100
462/462 [==============================] - 0s 318us/step - loss: 3.0246 - accuracy: 0.6190 - val_loss: 0.9642 - val_accuracy: 0.7323
Epoch 6/100
462/462 [==============================] - 0s 371us/step - loss: 2.8984 - accuracy: 0.6732 - val_loss: 1.2928 - val_accuracy: 0.7677
Epoch 7/100
462/462 [==============================] - 0s 416us/step - loss: 2.8975 - 

Epoch 57/100
462/462 [==============================] - 0s 340us/step - loss: 0.8509 - accuracy: 0.7879 - val_loss: 1.0065 - val_accuracy: 0.9343
Epoch 58/100
462/462 [==============================] - 0s 310us/step - loss: 1.0456 - accuracy: 0.7727 - val_loss: 0.9897 - val_accuracy: 0.9293
Epoch 59/100
462/462 [==============================] - 0s 318us/step - loss: 0.9309 - accuracy: 0.7576 - val_loss: 1.0163 - val_accuracy: 0.9293
Epoch 60/100
462/462 [==============================] - 0s 389us/step - loss: 0.8784 - accuracy: 0.7641 - val_loss: 1.5296 - val_accuracy: 0.8990
Epoch 61/100
462/462 [==============================] - 0s 279us/step - loss: 0.9912 - accuracy: 0.7814 - val_loss: 0.9218 - val_accuracy: 0.9444
Epoch 62/100
462/462 [==============================] - 0s 308us/step - loss: 0.7651 - accuracy: 0.8117 - val_loss: 0.9188 - val_accuracy: 0.9343
Epoch 63/100
462/462 [==============================] - 0s 285us/step - loss: 0.6801 - accuracy: 0.7879 - val_loss: 0.6656 -

In [92]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

198/198 [==============================] - 0s 167us/step
over-sampling test accuracy: 94.44%


In [73]:
pred5 = model1_over5.predict_classes(X_test_over)
pred5

array([1, 2, 1, 0, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 2, 1, 2, 2, 1, 1,
       2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 1, 1, 2,
       0, 0, 1, 2, 2, 0, 0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 2, 2, 2, 2, 0, 2,
       2, 2, 0, 1, 2, 1, 0, 0, 0, 2, 0, 1, 1, 0, 0, 1, 0, 1, 2, 2, 0, 0,
       2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 2, 0, 2, 0, 1, 2, 0, 1, 1, 1, 2, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 0, 1, 0, 2, 1,
       1, 1, 1, 0, 1, 0, 2, 0, 0, 1, 1, 2, 0, 1, 0, 2, 1, 1, 1, 1, 0, 2,
       0, 1, 0, 0, 2, 1, 1, 1, 1, 2, 2, 1, 2, 0, 1, 0, 2, 1, 2, 0, 1, 2,
       2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 1, 1, 0, 2, 1, 0, 1, 0, 1, 0, 2, 1])

In [74]:
dat5['pred'] = pred5
dat5

,0,test,pred
0,NRS241,1,1
1,NRS148,2,2
2,NRS255,1,1
3,NRS214,0,0
4,NRS148,2,2
...,...,...,...
193,CFBRSa30,0,0
194,NRS266,1,1
195,SR4152,0,0
196,NRS109,2,2


In [75]:
proba5 = model1_over5.predict_proba(X_test_over)
dat_proba5 = pd.DataFrame(proba5)

In [76]:
dat_proba5

,0,1,2
0,1.010424e-03,9.989893e-01,2.661334e-07
1,1.171583e-14,1.380678e-15,1.000000e+00
2,6.208902e-13,1.000000e+00,2.522147e-14
3,1.000000e+00,1.010950e-12,7.298213e-14
4,1.171583e-14,1.380678e-15,1.000000e+00
...,...,...,...
193,1.000000e+00,3.208899e-15,1.537287e-16
194,6.152107e-09,1.000000e+00,1.621104e-10
195,1.000000e+00,1.045865e-11,8.918558e-13
196,4.125410e-12,1.115170e-12,1.000000e+00


In [77]:
dat_proba5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba5.csv", index = False,
         header=None)

In [78]:
dat5.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/5p002ypST.csv", index = False,
         header=None)

In [96]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 272us/step - loss: 0.6857 - accuracy: 0.8139 - val_loss: 0.7262 - val_accuracy: 0.9495
Epoch 2/100
462/462 [==============================] - 0s 237us/step - loss: 0.6256 - accuracy: 0.8030 - val_loss: 0.6635 - val_accuracy: 0.9495
Epoch 3/100
462/462 [==============================] - 0s 204us/step - loss: 0.5761 - accuracy: 0.8203 - val_loss: 0.5914 - val_accuracy: 0.9545
Epoch 4/100
462/462 [==============================] - 0s 210us/step - loss: 0.6384 - accuracy: 0.8160 - val_loss: 0.6722 - val_accuracy: 0.9545
Epoch 5/100
462/462 [==============================] - 0s 211us/step - loss: 0.5305 - accuracy: 0.7987 - val_loss: 0.6152 - val_accuracy: 0.9646
Epoch 6/100
462/462 [==============================] - 0s 210us/step - loss: 0.5755 - accuracy: 0.8074 - val_loss: 0.5865 - val_accuracy: 0.9646
Epoch 7/100
462/462 [==============================] - 0s 222us/step - loss: 0.5525 

Epoch 57/100
462/462 [==============================] - 0s 195us/step - loss: 0.5862 - accuracy: 0.7900 - val_loss: 0.5155 - val_accuracy: 0.9747
Epoch 58/100
462/462 [==============================] - 0s 191us/step - loss: 0.5286 - accuracy: 0.8333 - val_loss: 0.8280 - val_accuracy: 0.9343
Epoch 59/100
462/462 [==============================] - 0s 197us/step - loss: 0.4949 - accuracy: 0.8182 - val_loss: 0.4816 - val_accuracy: 0.9697
Epoch 60/100
462/462 [==============================] - 0s 189us/step - loss: 0.4919 - accuracy: 0.8030 - val_loss: 0.6138 - val_accuracy: 0.9545
Epoch 61/100
462/462 [==============================] - 0s 188us/step - loss: 0.5404 - accuracy: 0.7857 - val_loss: 0.8611 - val_accuracy: 0.9394
Epoch 62/100
462/462 [==============================] - 0s 201us/step - loss: 0.6215 - accuracy: 0.7857 - val_loss: 0.6417 - val_accuracy: 0.9596
Epoch 63/100
462/462 [==============================] - 0s 222us/step - loss: 0.4658 - accuracy: 0.7879 - val_loss: 0.4440 -

In [97]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 79.92%


In [152]:
df_proba5 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [153]:
df_proba5

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS241,1,1,1.342914e-03,9.986569e-01,2.348628e-07
1,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
2,p002ykpresabsSTCC_qual,NRS255,1,1,1.780311e-07,9.999999e-01,2.544841e-12
3,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,2.203547e-10,5.688883e-15
4,p002ykpresabsSTCC_qual,NRS148,2,2,5.170289e-08,1.017893e-07,9.999999e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,BCH-SA-12,0,0,1.000000e+00,1.152503e-09,1.898730e-09
1978,pyopresabsSTCC_qual,NRS049,0,1,8.401357e-11,1.000000e+00,3.209735e-13
1979,pyopresabsSTCC_qual,NRS022,0,0,1.000000e+00,4.755084e-10,1.974275e-10
1980,pyopresabsSTCC_qual,NRS236,1,1,1.357345e-08,1.000000e+00,1.293117e-10


In [154]:
y_prob5 = df_proba5[df_proba5['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob5 = y_prob5.to_numpy()
y_prob5

array([[1.01042390e-03, 9.98989300e-01, 2.66133360e-07],
       [1.17158345e-14, 1.38067790e-15, 1.00000000e+00],
       [6.20890200e-13, 1.00000000e+00, 2.52214660e-14],
       [1.00000000e+00, 1.01095010e-12, 7.29821300e-14],
       [1.17158345e-14, 1.38067790e-15, 1.00000000e+00],
       [6.59399170e-06, 9.99993440e-01, 1.21551270e-09],
       [4.12541800e-12, 1.11517380e-12, 1.00000000e+00],
       [2.42376170e-11, 8.64185900e-12, 1.00000000e+00],
       [7.05474650e-08, 9.99999900e-01, 3.39856570e-09],
       [3.29710060e-05, 9.99967000e-01, 8.31547100e-11],
       [1.17158345e-14, 1.38067790e-15, 1.00000000e+00],
       [2.42376170e-11, 8.64185900e-12, 1.00000000e+00],
       [4.12541800e-12, 1.11517380e-12, 1.00000000e+00],
       [8.64502100e-09, 1.00000000e+00, 2.47795400e-10],
       [1.00000000e+00, 6.37299230e-24, 7.31899400e-26],
       [4.12541800e-12, 1.11517380e-12, 1.00000000e+00],
       [1.17158345e-14, 1.38067790e-15, 1.00000000e+00],
       [2.98721130e-05, 9.99965

In [155]:
ovo5 = rocauc_ovo(y_test_over, y_prob5, average="macro", multi_class="ovo")
ovo5

0.9891720232629323

In [156]:
ovr5 = rocauc_ovr(y_test_over, y_prob5, average="macro", multi_class="ovr")
ovr5

0.9891720232629323

In [157]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=678,
                                                    stratify=y_over)

In [158]:
dat6 = pd.DataFrame(X_test_over[:,0])
dat6['test'] = y_test_over

In [159]:
dat6

,0,test
0,NRS209,2
1,NRS386,1
2,NRS148,2
3,NRS178,0
4,NRS237,0
...,...,...
193,NRS209,2
194,NRS002,0
195,NRS109,2
196,BCH-SA-03,1


In [160]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [83]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [84]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 1s 1ms/step - loss: 6.7380 - accuracy: 0.4113 - val_loss: 1.7001 - val_accuracy: 0.6667
Epoch 2/100
462/462 [==============================] - 0s 474us/step - loss: 4.3364 - accuracy: 0.5195 - val_loss: 1.1548 - val_accuracy: 0.5505
Epoch 3/100
462/462 [==============================] - 0s 408us/step - loss: 4.1968 - accuracy: 0.5758 - val_loss: 0.9219 - val_accuracy: 0.7020
Epoch 4/100
462/462 [==============================] - 0s 262us/step - loss: 4.0467 - accuracy: 0.5931 - val_loss: 0.8227 - val_accuracy: 0.7727
Epoch 5/100
462/462 [==============================] - 0s 190us/step - loss: 3.4063 - accuracy: 0.6450 - val_loss: 0.7877 - val_accuracy: 0.7576
Epoch 6/100
462/462 [==============================] - 0s 216us/step - loss: 3.5021 - accuracy: 0.6450 - val_loss: 0.7555 - val_accuracy: 0.7626
Epoch 7/100
462/462 [==============================] - 0s 343us/step - loss: 3.8422 - 

Epoch 57/100
462/462 [==============================] - 0s 202us/step - loss: 1.1678 - accuracy: 0.7489 - val_loss: 0.8501 - val_accuracy: 0.9091
Epoch 58/100
462/462 [==============================] - 0s 318us/step - loss: 1.2515 - accuracy: 0.7424 - val_loss: 0.8911 - val_accuracy: 0.9091
Epoch 59/100
462/462 [==============================] - 0s 254us/step - loss: 1.2916 - accuracy: 0.7316 - val_loss: 0.6882 - val_accuracy: 0.8838
Epoch 60/100
462/462 [==============================] - 0s 240us/step - loss: 1.2354 - accuracy: 0.7597 - val_loss: 0.8152 - val_accuracy: 0.9293
Epoch 61/100
462/462 [==============================] - 0s 270us/step - loss: 1.1199 - accuracy: 0.7727 - val_loss: 0.7966 - val_accuracy: 0.9394
Epoch 62/100
462/462 [==============================] - 0s 234us/step - loss: 1.1686 - accuracy: 0.7706 - val_loss: 0.7746 - val_accuracy: 0.8939
Epoch 63/100
462/462 [==============================] - 0s 195us/step - loss: 1.1383 - accuracy: 0.7662 - val_loss: 0.8289 -

In [109]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

198/198 [==============================] - 0s 314us/step
over-sampling test accuracy: 94.44%


In [86]:
pred6 = model1_over6.predict_classes(X_test_over)
pred6

array([2, 1, 2, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0,
       0, 0, 1, 2, 1, 2, 2, 0, 1, 0, 2, 2, 1, 0, 0, 2, 0, 2, 1, 2, 1, 2,
       2, 2, 2, 0, 0, 1, 2, 0, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 0, 1, 2, 2,
       0, 0, 1, 2, 1, 0, 1, 2, 0, 1, 0, 2, 1, 2, 2, 1, 0, 2, 0, 1, 0, 0,
       0, 2, 2, 0, 1, 0, 2, 0, 0, 1, 2, 1, 1, 1, 2, 0, 1, 2, 2, 1, 0, 1,
       2, 1, 0, 2, 1, 1, 2, 1, 2, 1, 1, 1, 0, 2, 2, 1, 2, 0, 1, 0, 1, 0,
       1, 1, 2, 0, 2, 2, 0, 1, 0, 1, 1, 0, 2, 0, 2, 1, 0, 2, 0, 2, 1, 2,
       1, 2, 1, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1,
       0, 1, 1, 0, 0, 2, 1, 0, 1, 1, 1, 0, 2, 1, 2, 2, 1, 2, 0, 2, 1, 1])

In [87]:
dat6['pred'] = pred6
dat6

,0,test,pred
0,NRS209,2,2
1,NRS386,1,1
2,NRS148,2,2
3,NRS178,0,1
4,NRS237,0,0
...,...,...,...
193,NRS209,2,2
194,NRS002,0,0
195,NRS109,2,2
196,BCH-SA-03,1,1


In [88]:
proba6 = model1_over6.predict_proba(X_test_over)
dat_proba6 = pd.DataFrame(proba6)

In [89]:
dat_proba6

,0,1,2
0,2.090142e-08,2.289380e-08,1.000000e+00
1,2.581215e-04,9.997348e-01,6.993341e-06
2,7.165129e-09,2.305962e-09,1.000000e+00
3,4.373221e-12,1.000000e+00,4.699599e-16
4,1.000000e+00,2.403704e-10,4.481151e-11
...,...,...,...
193,2.090142e-08,2.289380e-08,1.000000e+00
194,8.901500e-01,1.098500e-01,7.356455e-09
195,1.026436e-08,6.748064e-08,9.999999e-01
196,2.584839e-07,9.999998e-01,4.213840e-13


In [90]:
dat_proba6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba6.csv", index = False,
         header=None)

In [91]:
dat6.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/6p002ypST.csv", index = False,
         header=None)

In [113]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 386us/step - loss: 0.6731 - accuracy: 0.7900 - val_loss: 0.7538 - val_accuracy: 0.9444
Epoch 2/100
462/462 [==============================] - 0s 256us/step - loss: 1.0901 - accuracy: 0.7684 - val_loss: 0.7902 - val_accuracy: 0.9545
Epoch 3/100
462/462 [==============================] - 0s 275us/step - loss: 0.8114 - accuracy: 0.7727 - val_loss: 0.9618 - val_accuracy: 0.9192
Epoch 4/100
462/462 [==============================] - 0s 278us/step - loss: 0.6523 - accuracy: 0.7835 - val_loss: 0.7545 - val_accuracy: 0.9444
Epoch 5/100
462/462 [==============================] - 0s 314us/step - loss: 0.5976 - accuracy: 0.7835 - val_loss: 0.8315 - val_accuracy: 0.9495
Epoch 6/100
462/462 [==============================] - 0s 288us/step - loss: 0.6995 - accuracy: 0.7771 - val_loss: 0.8870 - val_accuracy: 0.9444
Epoch 7/100
462/462 [==============================] - 0s 290us/step - loss: 0.6536 

Epoch 57/100
462/462 [==============================] - 0s 201us/step - loss: 0.4255 - accuracy: 0.7900 - val_loss: 0.7940 - val_accuracy: 0.9545
Epoch 58/100
462/462 [==============================] - 0s 188us/step - loss: 0.4993 - accuracy: 0.8117 - val_loss: 0.7524 - val_accuracy: 0.9545
Epoch 59/100
462/462 [==============================] - 0s 184us/step - loss: 0.4064 - accuracy: 0.7944 - val_loss: 0.8227 - val_accuracy: 0.9444
Epoch 60/100
462/462 [==============================] - 0s 196us/step - loss: 0.4458 - accuracy: 0.7965 - val_loss: 0.7324 - val_accuracy: 0.9545
Epoch 61/100
462/462 [==============================] - 0s 220us/step - loss: 0.4283 - accuracy: 0.8052 - val_loss: 0.8021 - val_accuracy: 0.9495
Epoch 62/100
462/462 [==============================] - 0s 199us/step - loss: 0.3975 - accuracy: 0.8095 - val_loss: 0.7431 - val_accuracy: 0.9545
Epoch 63/100
462/462 [==============================] - 0s 207us/step - loss: 0.3631 - accuracy: 0.8333 - val_loss: 0.7710 -

In [114]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 79.57%


In [161]:
df_proba6 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [162]:
df_proba6

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,1.790400e-08,4.141849e-08,1.000000e+00
1,p002ykpresabsSTCC_qual,NRS386,1,1,5.739934e-04,9.994259e-01,6.773014e-08
2,p002ykpresabsSTCC_qual,NRS148,2,2,5.286934e-09,1.269109e-08,1.000000e+00
3,p002ykpresabsSTCC_qual,NRS178,0,1,6.494936e-12,1.000000e+00,2.537080e-25
4,p002ykpresabsSTCC_qual,NRS237,0,1,5.701098e-02,9.399204e-01,3.068583e-03
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS272,0,0,9.999607e-01,3.367024e-05,5.776848e-06
1978,pyopresabsSTCC_qual,NRS112,1,1,8.275442e-08,9.999999e-01,3.739556e-09
1979,pyopresabsSTCC_qual,NRS064,1,1,2.168245e-08,1.000000e+00,9.603962e-09
1980,pyopresabsSTCC_qual,BCH-SA-04,0,0,1.000000e+00,1.026408e-15,1.630406e-14


In [163]:
y_prob6 = df_proba6[df_proba6['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob6 = y_prob6.to_numpy()
y_prob6

array([[2.09014200e-08, 2.28938020e-08, 1.00000000e+00],
       [2.58121460e-04, 9.99734800e-01, 6.99334120e-06],
       [7.16512900e-09, 2.30596230e-09, 1.00000000e+00],
       [4.37322050e-12, 1.00000000e+00, 4.69959900e-16],
       [1.00000000e+00, 2.40370420e-10, 4.48115120e-11],
       [6.21077400e-05, 9.99937900e-01, 7.19250600e-09],
       [2.58483650e-07, 9.99999760e-01, 4.21383980e-13],
       [2.09014200e-08, 2.28938020e-08, 1.00000000e+00],
       [6.21077400e-05, 9.99937900e-01, 7.19250600e-09],
       [1.11004330e-01, 8.67309500e-01, 2.16861300e-02],
       [2.58121460e-04, 9.99734800e-01, 6.99334120e-06],
       [8.98108850e-07, 9.99999050e-01, 5.97431840e-09],
       [1.75746940e-08, 1.00000000e+00, 3.12505580e-11],
       [1.34828690e-02, 9.86517200e-01, 9.37456000e-10],
       [1.00000000e+00, 9.04878800e-19, 1.96243880e-20],
       [2.58483650e-07, 9.99999760e-01, 4.21383980e-13],
       [1.00000000e+00, 1.55860100e-11, 1.04412880e-14],
       [1.30124600e-06, 9.99998

In [164]:
ovo6 = rocauc_ovo(y_test_over, y_prob6, average="macro", multi_class="ovo")
ovo6

0.9815388735843281

In [165]:
ovr6 = rocauc_ovr(y_test_over, y_prob6, average="macro", multi_class="ovr")
ovr6

0.9815388735843281

In [166]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=789,
                                                    stratify=y_over)

In [167]:
dat7 = pd.DataFrame(X_test_over[:,0])
dat7['test'] = y_test_over

In [168]:
dat7

,0,test
0,NRS209,2
1,BCH-SA-09,1
2,NRS224,0
3,NRS209,2
4,NRS235,1
...,...,...
193,NRS209,2
194,CFBREBSa131,1
195,CFBREBSa103,0
196,NRS188,1


In [169]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [96]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [97]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [98]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 757us/step - loss: 6.8590 - accuracy: 0.4307 - val_loss: 2.7388 - val_accuracy: 0.5960
Epoch 2/100
462/462 [==============================] - 0s 254us/step - loss: 4.3318 - accuracy: 0.6342 - val_loss: 1.8825 - val_accuracy: 0.7475
Epoch 3/100
462/462 [==============================] - 0s 349us/step - loss: 3.7735 - accuracy: 0.6861 - val_loss: 1.3884 - val_accuracy: 0.6970
Epoch 4/100
462/462 [==============================] - 0s 253us/step - loss: 3.4658 - accuracy: 0.6688 - val_loss: 1.4580 - val_accuracy: 0.7576
Epoch 5/100
462/462 [==============================] - 0s 267us/step - loss: 2.9221 - accuracy: 0.7056 - val_loss: 1.1876 - val_accuracy: 0.7020
Epoch 6/100
462/462 [==============================] - 0s 298us/step - loss: 3.7692 - accuracy: 0.6623 - val_loss: 1.2398 - val_accuracy: 0.7374
Epoch 7/100
462/462 [==============================] - 0s 253us/step - loss: 3.5747 

Epoch 57/100
462/462 [==============================] - 0s 373us/step - loss: 0.9577 - accuracy: 0.7597 - val_loss: 0.9715 - val_accuracy: 0.9242
Epoch 58/100
462/462 [==============================] - 0s 247us/step - loss: 0.8968 - accuracy: 0.7749 - val_loss: 0.9845 - val_accuracy: 0.9394
Epoch 59/100
462/462 [==============================] - 0s 351us/step - loss: 0.9281 - accuracy: 0.8052 - val_loss: 1.1307 - val_accuracy: 0.9141
Epoch 60/100
462/462 [==============================] - 0s 308us/step - loss: 0.9591 - accuracy: 0.7727 - val_loss: 1.0140 - val_accuracy: 0.9192
Epoch 61/100
462/462 [==============================] - 0s 265us/step - loss: 0.9073 - accuracy: 0.7662 - val_loss: 1.1590 - val_accuracy: 0.9242
Epoch 62/100
462/462 [==============================] - 0s 257us/step - loss: 0.8910 - accuracy: 0.7835 - val_loss: 1.0454 - val_accuracy: 0.9293
Epoch 63/100
462/462 [==============================] - 0s 248us/step - loss: 0.8438 - accuracy: 0.7900 - val_loss: 0.9608 -

In [122]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))

198/198 [==============================] - 0s 196us/step
over-sampling test accuracy: 91.41%


In [99]:
pred7 = model1_over7.predict_classes(X_test_over)
pred7

array([2, 1, 0, 2, 1, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 2, 2, 0, 2, 1,
       2, 0, 0, 2, 1, 0, 1, 0, 1, 2, 1, 2, 2, 2, 1, 2, 0, 0, 2, 1, 1, 0,
       2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2, 0, 2, 0, 0, 0, 0,
       0, 2, 0, 2, 0, 1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 2, 0, 2, 1, 0, 1, 1, 2, 1,
       1, 0, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 0, 1, 2, 1, 2, 0, 0, 0,
       2, 2, 1, 1, 0, 1, 0, 1, 2, 0, 0, 1, 2, 0, 1, 0, 1, 0, 1, 0, 1, 2,
       0, 2, 2, 0, 0, 1, 2, 1, 0, 2, 1, 2, 1, 1, 2, 1, 2, 0, 2, 2, 0, 2,
       0, 0, 2, 0, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 1, 0, 1, 2, 1, 0, 1, 2])

In [100]:
dat7['pred'] = pred7
dat7

,0,test,pred
0,NRS209,2,2
1,BCH-SA-09,1,1
2,NRS224,0,0
3,NRS209,2,2
4,NRS235,1,1
...,...,...,...
193,NRS209,2,2
194,CFBREBSa131,1,1
195,CFBREBSa103,0,0
196,NRS188,1,1


In [101]:
proba7 = model1_over7.predict_proba(X_test_over)
dat_proba7 = pd.DataFrame(proba7)

In [102]:
dat_proba7

,0,1,2
0,9.617234e-09,1.121028e-08,1.000000e+00
1,2.367589e-05,9.999763e-01,3.152567e-08
2,1.000000e+00,1.816863e-14,4.248027e-12
3,9.617234e-09,1.121028e-08,1.000000e+00
4,8.324296e-08,9.999999e-01,1.054044e-09
...,...,...,...
193,9.617215e-09,1.121026e-08,1.000000e+00
194,7.296561e-02,8.494496e-01,7.758473e-02
195,1.000000e+00,7.829268e-15,1.843543e-17
196,1.117732e-08,1.000000e+00,4.808631e-10


In [103]:
dat_proba7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba7.csv", index = False,
         header=None)

In [104]:
dat7.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/7p002ypST.csv", index = False,
         header=None)

In [126]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 329us/step - loss: 0.9109 - accuracy: 0.7749 - val_loss: 1.0429 - val_accuracy: 0.8990
Epoch 2/100
462/462 [==============================] - 0s 309us/step - loss: 0.8131 - accuracy: 0.7771 - val_loss: 1.0394 - val_accuracy: 0.8939
Epoch 3/100
462/462 [==============================] - 0s 300us/step - loss: 0.8217 - accuracy: 0.8182 - val_loss: 1.3902 - val_accuracy: 0.8889
Epoch 4/100
462/462 [==============================] - 0s 246us/step - loss: 0.8305 - accuracy: 0.7684 - val_loss: 0.8899 - val_accuracy: 0.9040
Epoch 5/100
462/462 [==============================] - 0s 208us/step - loss: 0.8940 - accuracy: 0.7619 - val_loss: 1.0573 - val_accuracy: 0.9091
Epoch 6/100
462/462 [==============================] - 0s 218us/step - loss: 0.8499 - accuracy: 0.7835 - val_loss: 1.0625 - val_accuracy: 0.9040
Epoch 7/100
462/462 [==============================] - 0s 216us/step - loss: 0.8009 

Epoch 57/100
462/462 [==============================] - 0s 213us/step - loss: 0.8016 - accuracy: 0.7792 - val_loss: 1.7716 - val_accuracy: 0.8788
Epoch 58/100
462/462 [==============================] - 0s 186us/step - loss: 0.6755 - accuracy: 0.7792 - val_loss: 1.3477 - val_accuracy: 0.8990
Epoch 59/100
462/462 [==============================] - 0s 203us/step - loss: 0.6580 - accuracy: 0.7922 - val_loss: 1.0636 - val_accuracy: 0.9242
Epoch 60/100
462/462 [==============================] - 0s 243us/step - loss: 0.6320 - accuracy: 0.7922 - val_loss: 1.3889 - val_accuracy: 0.9040
Epoch 61/100
462/462 [==============================] - 0s 211us/step - loss: 0.7383 - accuracy: 0.7554 - val_loss: 1.3223 - val_accuracy: 0.9040
Epoch 62/100
462/462 [==============================] - 0s 213us/step - loss: 0.5548 - accuracy: 0.8095 - val_loss: 1.5507 - val_accuracy: 0.8838
Epoch 63/100
462/462 [==============================] - 0s 189us/step - loss: 0.6508 - accuracy: 0.7835 - val_loss: 1.1494 -

In [127]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 78.82%


In [170]:
df_proba7 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [171]:
df_proba7

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
1,p002ykpresabsSTCC_qual,BCH-SA-09,1,1,1.137139e-06,9.999988e-01,2.067601e-09
2,p002ykpresabsSTCC_qual,NRS224,0,0,1.000000e+00,2.093110e-31,0.000000e+00
3,p002ykpresabsSTCC_qual,NRS209,2,2,8.300497e-12,1.036520e-09,1.000000e+00
4,p002ykpresabsSTCC_qual,NRS235,1,1,2.243513e-02,9.774035e-01,1.615106e-04
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS035,0,0,9.354528e-01,6.414209e-02,4.051121e-04
1978,pyopresabsSTCC_qual,NRS260,1,1,4.808470e-08,1.000000e+00,7.364639e-09
1979,pyopresabsSTCC_qual,CA9,0,0,1.000000e+00,2.361323e-08,2.871247e-08
1980,pyopresabsSTCC_qual,NRS183,1,1,2.755864e-07,9.999998e-01,5.310879e-08


In [172]:
y_prob7 = df_proba7[df_proba7['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob7 = y_prob7.to_numpy()
y_prob7

array([[9.61723400e-09, 1.12102800e-08, 1.00000000e+00],
       [2.36758900e-05, 9.99976300e-01, 3.15256700e-08],
       [1.00000000e+00, 1.81686280e-14, 4.24802700e-12],
       [9.61723400e-09, 1.12102800e-08, 1.00000000e+00],
       [8.32429600e-08, 9.99999900e-01, 1.05404420e-09],
       [4.62640420e-05, 9.99953600e-01, 9.47458100e-08],
       [7.88794700e-08, 9.99999900e-01, 6.59497940e-11],
       [9.61723400e-09, 1.12102800e-08, 1.00000000e+00],
       [7.29656140e-02, 8.49449630e-01, 7.75847300e-02],
       [8.32429600e-08, 9.99999900e-01, 1.05404420e-09],
       [2.63931080e-08, 1.00000000e+00, 1.31076380e-09],
       [1.00000000e+00, 1.02502680e-16, 6.67100450e-14],
       [1.00000000e+00, 6.40438040e-17, 7.43121200e-20],
       [1.00000000e+00, 5.07298100e-14, 1.57200220e-16],
       [7.88794700e-08, 9.99999900e-01, 6.59497940e-11],
       [1.54003670e-08, 1.82290110e-08, 1.00000000e+00],
       [7.29656140e-02, 8.49449630e-01, 7.75847300e-02],
       [9.61723400e-09, 1.12102

In [173]:
ovo7 = rocauc_ovo(y_test_over, y_prob7, average="macro", multi_class="ovo")
ovo7

0.9852502295684115

In [174]:
ovr7 = rocauc_ovr(y_test_over, y_prob7, average="macro", multi_class="ovr")
ovr7

0.9852502295684115

In [175]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=890,
                                                    stratify=y_over)

In [176]:
dat8 = pd.DataFrame(X_test_over[:,0])
dat8['test'] = y_test_over

In [177]:
dat8

,0,test
0,CFBREBSa116,0
1,NRS214,0
2,NRS148,2
3,NRS148,2
4,NRS148,2
...,...,...
193,NRS148,2
194,NRS054,0
195,NRS109,2
196,NRS216,1


In [178]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [109]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [110]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 788us/step - loss: 6.2071 - accuracy: 0.5195 - val_loss: 2.4389 - val_accuracy: 0.7323
Epoch 2/100
462/462 [==============================] - 0s 320us/step - loss: 4.3776 - accuracy: 0.5823 - val_loss: 1.4943 - val_accuracy: 0.6061
Epoch 3/100
462/462 [==============================] - 0s 502us/step - loss: 3.6823 - accuracy: 0.6364 - val_loss: 1.0597 - val_accuracy: 0.7677
Epoch 4/100
462/462 [==============================] - 0s 286us/step - loss: 3.6903 - accuracy: 0.6407 - val_loss: 1.0230 - val_accuracy: 0.7576
Epoch 5/100
462/462 [==============================] - 0s 401us/step - loss: 3.3820 - accuracy: 0.6602 - val_loss: 0.9743 - val_accuracy: 0.7727
Epoch 6/100
462/462 [==============================] - 0s 399us/step - loss: 3.3565 - accuracy: 0.6667 - val_loss: 1.1421 - val_accuracy: 0.7424
Epoch 7/100
462/462 [==============================] - 0s 388us/step - loss: 3.2720 

Epoch 57/100
462/462 [==============================] - 0s 281us/step - loss: 0.9360 - accuracy: 0.7835 - val_loss: 0.8982 - val_accuracy: 0.8990
Epoch 58/100
462/462 [==============================] - 0s 305us/step - loss: 1.0471 - accuracy: 0.7424 - val_loss: 0.6354 - val_accuracy: 0.9394
Epoch 59/100
462/462 [==============================] - 0s 372us/step - loss: 0.8424 - accuracy: 0.7987 - val_loss: 1.0164 - val_accuracy: 0.8939
Epoch 60/100
462/462 [==============================] - 0s 332us/step - loss: 0.8735 - accuracy: 0.7922 - val_loss: 0.6414 - val_accuracy: 0.9394
Epoch 61/100
462/462 [==============================] - 0s 241us/step - loss: 0.9453 - accuracy: 0.7771 - val_loss: 0.7540 - val_accuracy: 0.9293
Epoch 62/100
462/462 [==============================] - 0s 225us/step - loss: 0.7696 - accuracy: 0.7835 - val_loss: 0.8552 - val_accuracy: 0.8990
Epoch 63/100
462/462 [==============================] - 0s 411us/step - loss: 0.8993 - accuracy: 0.7749 - val_loss: 1.0912 -

In [135]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))

198/198 [==============================] - 0s 177us/step
over-sampling test accuracy: 94.44%


In [112]:
pred8 = model1_over8.predict_classes(X_test_over)
pred8

array([0, 0, 2, 2, 2, 1, 2, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 2, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 2, 1, 0, 2, 1, 0, 1, 1, 2, 2, 0, 0, 0, 2, 0, 1, 2,
       2, 2, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 2, 1, 0, 0, 1, 2, 0, 2, 2,
       1, 2, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 2, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 2, 0, 2, 0, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 0,
       2, 1, 1, 2, 2, 2, 0, 1, 1, 2, 2, 1, 0, 2, 2, 1, 1, 2, 2, 2, 2, 0,
       0, 2, 2, 1, 1, 0, 1, 2, 1, 0, 0, 1, 2, 0, 0, 2, 1, 0, 0, 1, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 2, 0, 0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 1, 0, 1, 2, 0, 1, 2, 0, 2, 0, 2, 1, 1])

In [113]:
dat8['pred'] = pred8
dat8

,0,test,pred
0,CFBREBSa116,0,0
1,NRS214,0,0
2,NRS148,2,2
3,NRS148,2,2
4,NRS148,2,2
...,...,...,...
193,NRS148,2,2
194,NRS054,0,0
195,NRS109,2,2
196,NRS216,1,1


In [114]:
proba8 = model1_over8.predict_proba(X_test_over)
dat_proba8 = pd.DataFrame(proba8)

In [115]:
dat_proba8

,0,1,2
0,1.000000e+00,2.989959e-17,2.267019e-16
1,1.000000e+00,7.571602e-11,6.163611e-11
2,1.903322e-11,9.252212e-12,1.000000e+00
3,1.903322e-11,9.252212e-12,1.000000e+00
4,1.903322e-11,9.252212e-12,1.000000e+00
...,...,...,...
193,1.903325e-11,9.252230e-12,1.000000e+00
194,9.999892e-01,7.644894e-06,3.258920e-06
195,3.963342e-13,1.159683e-13,1.000000e+00
196,9.209495e-09,1.000000e+00,1.830634e-10


In [116]:
dat_proba8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba8.csv", index = False,
         header=None)

In [117]:
dat8.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/8p002ypST.csv", index = False,
         header=None)

In [139]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 462 samples, validate on 198 samples
Epoch 1/100
462/462 [==============================] - 0s 274us/step - loss: 0.7292 - accuracy: 0.7965 - val_loss: 0.4938 - val_accuracy: 0.9444
Epoch 2/100
462/462 [==============================] - 0s 249us/step - loss: 0.6321 - accuracy: 0.7944 - val_loss: 0.4546 - val_accuracy: 0.9394
Epoch 3/100
462/462 [==============================] - 0s 238us/step - loss: 0.7012 - accuracy: 0.7792 - val_loss: 0.4362 - val_accuracy: 0.9444
Epoch 4/100
462/462 [==============================] - 0s 216us/step - loss: 0.8464 - accuracy: 0.7835 - val_loss: 0.6047 - val_accuracy: 0.9495
Epoch 5/100
462/462 [==============================] - 0s 263us/step - loss: 0.6445 - accuracy: 0.7879 - val_loss: 0.5600 - val_accuracy: 0.9495
Epoch 6/100
462/462 [==============================] - 0s 235us/step - loss: 0.7249 - accuracy: 0.7814 - val_loss: 0.7505 - val_accuracy: 0.9394
Epoch 7/100
462/462 [==============================] - 0s 288us/step - loss: 0.7690 

Epoch 57/100
462/462 [==============================] - 0s 223us/step - loss: 0.4636 - accuracy: 0.7749 - val_loss: 1.0269 - val_accuracy: 0.9293
Epoch 58/100
462/462 [==============================] - 0s 204us/step - loss: 0.4678 - accuracy: 0.8095 - val_loss: 0.5746 - val_accuracy: 0.9545
Epoch 59/100
462/462 [==============================] - 0s 226us/step - loss: 0.3769 - accuracy: 0.8139 - val_loss: 0.8104 - val_accuracy: 0.9444
Epoch 60/100
462/462 [==============================] - 0s 223us/step - loss: 0.4163 - accuracy: 0.8117 - val_loss: 0.5625 - val_accuracy: 0.9545
Epoch 61/100
462/462 [==============================] - 0s 325us/step - loss: 0.4630 - accuracy: 0.8095 - val_loss: 0.6194 - val_accuracy: 0.9545
Epoch 62/100
462/462 [==============================] - 0s 271us/step - loss: 0.4549 - accuracy: 0.7814 - val_loss: 0.7325 - val_accuracy: 0.9495
Epoch 63/100
462/462 [==============================] - 0s 400us/step - loss: 0.4125 - accuracy: 0.8225 - val_loss: 0.4591 -

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.161343). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


462/462 [==============================] - 0s 504us/step - loss: 0.4814 - accuracy: 0.7771 - val_loss: 0.4297 - val_accuracy: 0.9596
Epoch 73/100
462/462 [==============================] - 0s 297us/step - loss: 0.4096 - accuracy: 0.8052 - val_loss: 0.7078 - val_accuracy: 0.9444
Epoch 74/100
462/462 [==============================] - 0s 268us/step - loss: 0.4224 - accuracy: 0.8052 - val_loss: 0.8052 - val_accuracy: 0.9394
Epoch 75/100
462/462 [==============================] - 0s 263us/step - loss: 0.3643 - accuracy: 0.8074 - val_loss: 0.9018 - val_accuracy: 0.9343
Epoch 76/100
462/462 [==============================] - 0s 312us/step - loss: 0.4251 - accuracy: 0.8117 - val_loss: 0.8969 - val_accuracy: 0.9343
Epoch 77/100
462/462 [==============================] - 0s 235us/step - loss: 0.4515 - accuracy: 0.8074 - val_loss: 0.4287 - val_accuracy: 0.9545
Epoch 78/100
462/462 [==============================] - 0s 212us/step - loss: 0.4130 - accuracy: 0.8074 - val_loss: 0.7615 - val_accuracy

In [140]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 79.55%


In [179]:
df_proba8 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_regularizor_dropout_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [180]:
df_proba8

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabsSTCC_qual,CFBREBSa116,0,0,9.676203e-01,3.237956e-02,1.480166e-07
1,p002ykpresabsSTCC_qual,NRS214,0,0,1.000000e+00,6.534852e-11,2.250731e-18
2,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
3,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
4,p002ykpresabsSTCC_qual,NRS148,2,2,3.948571e-11,2.839096e-07,9.999998e-01
...,...,...,...,...,...,...,...
1977,pyopresabsSTCC_qual,NRS205,2,2,3.691095e-08,3.571927e-08,9.999999e-01
1978,pyopresabsSTCC_qual,CFBREBSa122,0,1,9.261665e-02,9.073822e-01,1.162373e-06
1979,pyopresabsSTCC_qual,NRS001,1,1,4.174278e-07,9.999995e-01,3.254024e-09
1980,pyopresabsSTCC_qual,NRS148,2,2,3.234670e-08,3.121212e-08,9.999999e-01


In [181]:
y_prob8 = df_proba8[df_proba8['phage']=='p002ypresabsSTCC_qual'].iloc[:,-3:]
y_prob8 = y_prob8.to_numpy()
y_prob8

array([[1.00000000e+00, 2.98995880e-17, 2.26701880e-16],
       [1.00000000e+00, 7.57160200e-11, 6.16361060e-11],
       [1.90332160e-11, 9.25221200e-12, 1.00000000e+00],
       [1.90332160e-11, 9.25221200e-12, 1.00000000e+00],
       [1.90332160e-11, 9.25221200e-12, 1.00000000e+00],
       [9.20947800e-09, 1.00000000e+00, 1.83062750e-10],
       [1.90332160e-11, 9.25221200e-12, 1.00000000e+00],
       [1.00000000e+00, 3.42661080e-14, 1.20149440e-13],
       [1.00000000e+00, 2.47253750e-11, 2.14327900e-11],
       [5.53588500e-09, 1.00000000e+00, 7.43924440e-11],
       [1.00000000e+00, 2.66637300e-14, 2.73315330e-13],
       [3.96337230e-13, 1.15969180e-13, 1.00000000e+00],
       [5.53588500e-09, 1.00000000e+00, 7.43924440e-11],
       [1.00000000e+00, 6.36831000e-14, 7.71469950e-14],
       [7.14991730e-10, 1.00000000e+00, 3.25342270e-13],
       [4.70374100e-09, 4.20116160e-09, 1.00000000e+00],
       [7.93568000e-02, 8.78490870e-01, 4.21523230e-02],
       [1.90332160e-11, 9.25221

In [182]:
ovo8 = rocauc_ovo(y_test_over, y_prob8, average="macro", multi_class="ovo")
ovo8

0.977502295684114

In [183]:
ovr8 = rocauc_ovr(y_test_over, y_prob8, average="macro", multi_class="ovr")
ovr8

0.977502295684114

In [186]:
ovos2 = [ovo5, ovo6, ovo7, ovo8]
np.mean(ovos2)

0.9833658555249465

In [187]:
np.std(ovos2)

0.004329597162429823

In [188]:
ovrs2 = [ovr5, ovr6, ovr7, ovr8]
np.mean(ovrs2)

0.9833658555249465

In [189]:
np.std(ovrs2)

0.004329597162429823

In [141]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [142]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 93.69%


In [143]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.013121584513306875


In [144]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [145]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 79.46%


In [146]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.0039923233
